In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import math
import glidertools as gt
import cmocean.cm as cmo
import matplotlib.pyplot as plt

from esdglider import gcp, glider, plots, utils
from esdglider.plots import adj_var, sci_vars, adj_var_label, label_size

deployment_info = {
    "deployment_name": "amlr03-20231128",
    "mode": "delayed",
}
# deployment_info = {
#     "deployment_name": 'amlr08-20220513', 
#     "mode": 'delayed', 
# }

# project = deployment_info["project"]
# year = utils.year_path(project, deployment)
bucket_name = 'amlr-gliders-deployments-dev'
deployments_path = f"/home/sam_woodman_noaa_gov/{bucket_name}"
config_path = f"/home/sam_woodman_noaa_gov/glider-lab/deployment-configs"

gcp.gcs_mount_bucket(
    "amlr-gliders-deployments-dev", deployments_path, 
    ro=False)

deployment_info["deploymentyaml"] = os.path.join(
    config_path, f"{deployment_info["deployment_name"]}.yml")

paths = glider.get_path_glider(deployment_info, deployments_path)

In [ ]:
ds_raw = xr.load_dataset(paths["tsrawpath"])
ds_eng = xr.load_dataset(paths["tsengpath"])
ds_sci = xr.load_dataset(paths["tsscipath"])
ds_g5 = xr.load_dataset(paths["gr5path"])

## Misc checks and work

In [ ]:
ds = ds_g5
var = "chlorophyll"

ds = ds.dropna(dim="depth", how="all")

fig, ax = plt.subplots(figsize=(11, 8.5))
std = np.nanstd(ds[var])
mean = np.nanmean(ds[var])

p1 = ax.pcolormesh(ds.time, ds.depth, adj_var(ds, var), cmap=sci_vars[var])
fig.colorbar(p1).set_label(label=adj_var_label(ds, var), size=label_size)
ax.invert_yaxis()

ax.set_title(
    # f"Deployment {deployment} for project {}\n std={std:0.2f} mean={mean:0.2f}",
    f"Deployment g for project gg\n std={std:0.2f} mean={mean:0.2f}",
    size=14,
)  # use set_title so that title is centered over the plot
ax.set_xlabel("Time", size=label_size)
ax.set_ylabel("Depth [m]", size=label_size)
# t = ax.text(0, -0.18, caption, horizontalalignment='left', verticalalignment='center', transform=ax.transAxes, wrap=True)

# for label in ax.get_xticklabels(which='major'):
#     label.set(rotation=15, horizontalalignment='center')
fig.autofmt_xdate()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(ds_eng.gps_longitude, ds_eng.gps_latitude, s=3)

# # Labels and title
# ax.set_xlabel("Longitude", size=label_size)
# ax.set_ylabel("Latitude", size=label_size)
# ax.set_title(title_str, size=title_size)
ax.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(ds_raw.gps_longitude, ds_raw.gps_latitude, s=3)

# # Labels and title
# ax.set_xlabel("Longitude", size=label_size)
# ax.set_ylabel("Latitude", size=label_size)
# ax.set_title(title_str, size=title_size)
ax.grid(True)
plt.show()

### Eng tvt


In [ ]:
eng_dict = plots.eng_plots_to_make(ds_raw)
print(eng_dict.keys())

In [ ]:
plots.eng_tvt_plot("diveEnergy", ds_raw, eng_dict)

In [ ]:
key = "diveEnergy"
# key = "leakDetect"
ds = ds_raw

deployment = ds.deployment_name

fig, ax = plt.subplots(figsize=(8.5, 8.5))


for i in range(len(eng_dict[key]["Y"])):
    if key == "oilVol":
        plot = ax.scatter(eng_dict[key]["X"], eng_dict[key]["Y"][i])
    else:
        plot = ax.scatter(
            eng_dict[key]["X"],
            eng_dict[key]["Y"][i],
            label=eng_dict[key]["Y"][i].name,
            c=eng_dict[key]["C"][i],
        )

    if eng_dict[key]["cb"] is not None:
        cbar = fig.colorbar(plot)
        cbar.set_label(eng_dict[key]["cb"])

ax.set_xlabel(eng_dict[key]["X"].name, size=label_size)
ax.set_ylabel(eng_dict[key]["Y"][0].name, size=label_size)

if len(eng_dict[key]["C"]) > 1:
    # ax.legend(loc="upper left")
    ax.legend() #loc="center right")

if eng_dict[key]["X"].name == "time":
    fig.autofmt_xdate()

plt.show()

## Test esdglider.plots functions

In [ ]:
ds = ds_sci

# Plot
fig, ax = plt.subplots(figsize=(8, 6))
# ax.scatter(ds.longitude, ds.latitude, s=3)
ax.scatter(
    ds.longitude, ds.latitude, s=3, 
    c=ds["profile_index"]
)

# Labels and title
ax.set_xlabel("Longitude", size=plots.label_size)
ax.set_ylabel("Latitude", size=plots.label_size)
ax.set_title("go go", size=plots.title_size)
ax.grid(True)

plt.show()

In [ ]:
# Glidertools plotting
plots.sci_timesection_gt_plot("density", ds_sci, show = False)

dat = ds_sci.where(ds_sci['profile_index'] % 1 == 0, drop = True)
x = dat.profile_index
y = dat.depth

fig, ax = plt.subplots(figsize=(11, 8.5))
ax = gt.plot(x, y, dat.density, cmap=cmo.dense, ax=ax, robust=True)\

gt.plot.bin_size(ds_sci.depth)

## Testing for making the surface map plots

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# from matplotlib_scalebar.scalebar import ScaleBar

ds = ds_g5
var = "profile_index"
crs = "Mercator"
figsize_x = 8.5
figsize_y = 11

# bar_file = "/home/sam_woodman_noaa_gov/ETOPO_2022_v1_15s_N45W135_erddap.nc"
# bar = xr.load_dataset(bar_file).rename({"latitude": "lat", "longitude": "lon"})
# bar = bar.where(bar.z <= 0, drop=True)

### Function
plots.sci_surface_map(
    "profile_index", 
    ds_g5, 
    crs=ccrs.Mercator(), 
    show=False, 
    bar=None, 
)

# # plt.show()

# ### Code
# deployment = ds.deployment_name
# start = ds.deployment_start[0:10]
# end = ds.deployment_end[0:10]

# map_lon_border = 0.1
# map_lat_border = 0.2
# glider_lon_min = ds.longitude.min()
# glider_lon_max = ds.longitude.max()
# glider_lat_min = ds.latitude.min()
# glider_lat_max = ds.latitude.max()

# # Using Cartopy
# if isinstance(crs, str):
#     crs = plots.crs_map(crs)
# fig, ax = plt.subplots(
#     figsize=(figsize_x, figsize_y),
#     subplot_kw={"projection": crs},
# )
# # ax = typing.cast(GeoAxes, ax)  # Explicitly cast to GeoAxes, for Pylance

# ax.set_xlabel("\n\n\nLongitude [Deg]", size=14)
# ax.set_ylabel("Latitude [Deg]\n\n\n", size=14)

# # Set extent of the map based on the glider data
# ax.set_extent(
#     [
#         glider_lon_min - map_lon_border,
#         glider_lon_max + 3 * map_lon_border,
#         glider_lat_min - map_lat_border,
#         glider_lat_max + map_lat_border,
#     ],
#     crs=ccrs.PlateCarree(),
# )

# # Add coastlines, countries, and continents
# # ax.coastlines(resolution="110m", linestyle="-", color="black")
# ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="#e0b479")
# ax.add_feature(cfeature.OCEAN, edgecolor="none", facecolor="#7bcbe3")

# # Add parallels and meridians; no scale bar since no built-in function
# gl = ax.gridlines(draw_labels=["bottom", "left"])  # type: ignore
# gl.xlabel_style = {"rotation": 15}

# # Special considerations for profile_index
# if var == "profile_index":
#     c = ds["profile"]
#     title = f"{deployment}: {var}\nfrom {start} to {end}"
#     cbar_label = "profile_index [1]"
# else:
#     c = ds[var].where(ds.depth <= 10, drop=True).mean(dim="depth")
#     title = f"{deployment}: 0 - 10m average {var}\nfrom {start} to {end}"
#     cbar_label = plots.adj_var_label(ds, var)

# p = ax.scatter(
#     ds.longitude,
#     ds.latitude,
#     c=c,
#     cmap=plots.sci_vars[var],
#     s=10,
#     zorder=2.5,
#     transform=ccrs.PlateCarree(),
# )

# # Add colorbar
# fig.colorbar(p, ax=ax, shrink=0.6, location="right").set_label(
#     label=cbar_label,
#     size=plots.label_size,
# )
# ax.set_title(title, size=plots.title_size)

# plt.show()

In [ ]:
ds[var].where(ds.depth <= 10, drop=True).mean(dim="depth")

In [ ]:
ds["profile"]

In [ ]:
ds_sci["depth"]